In [ ]:

import pandas as pd
import numpy as np
import plotly.express as px

import sklearn.model_selection as skm
from scipy.optimize import linprog

In [ ]:
data = pd.read_csv("data_banknote_authentication.txt")
cNames = ["variance","skewness","curtosis","entropy","isForgery"]
data.set_axis(cNames,axis=1,inplace=True)

In [ ]:
train,test = skm.train_test_split(data)
#75,25 ish

In [ ]:
significant_digits = 6
epsilon = 10**(-significant_digits)

m = ["highs-ds", "highs-ipm", "revised simplex", "interior-point"]

In [ ]:

train_no = train.loc[train["isForgery"]==1].iloc[:,[0,1,2,3]]
train_yes = train.loc[train["isForgery"]==0].iloc[:,[0,1,2,3]]
#split to X,Y by is forgery and cut the column out

In [ ]:
X = train_yes.to_numpy()
Y = train_no.to_numpy()

Xp = np.hstack((-X,np.ones(X.shape[0])[:,None]))
Xp = np.hstack((Xp,-np.eye(X.shape[0])))
Xp = np.hstack((Xp,np.zeros([X.shape[0],Y.shape[0]])))

Yp = np.hstack((Y,-np.ones(Y.shape[0])[:,None]))
Yp = np.hstack((Yp,np.zeros([Y.shape[0],X.shape[0]])))
Yp = np.hstack((Yp,-np.eye(Y.shape[0])))

A = np.vstack((Xp,Yp))
b = -np.ones(X.shape[0] + Y.shape[0])

c = np.array([0] * (X.shape[1] + 1) + [1]*X.shape[0] + [1]*Y.shape[0])

In [ ]:

for method in m:
    #maybe bounds should be (None, None) - not sure
    options = {"cholesky":False,"lstsq":True,"presolve":True,"sym_pos":False}
    
    result = linprog(c, A_ub=A, b_ub=b, method=method, bounds=((0,None)))
    #print(f"{method}:{result}")
    result = result.x
    try:
        result[abs(result) < epsilon] = 0

        a_result = result[0:4]
        b_result = result[5]
        print(f"{method}\na:{a_result}\nb:{b_result}")
    except:
        print(f"{method} found nothing")
        
#Every method produces same results 

highs-ds
a:[4.95887916 2.36301441 3.13103513 0.07929235]
b:0.0
highs-ipm
a:[4.95887916 2.36301441 3.13103513 0.07929235]
b:0.0
revised simplex
a:[4.95887916 2.36301441 3.13103513 0.07929235]
b:0.0
interior-point
a:[4.95887926 2.36301446 3.1310352  0.07929236]
b:0.0


In [ ]:
a = np.array([6.35473734,3.24645703,4.11899572,0.56123915])
b = 0.0
#counters
correctX = 0
correctY = 0
incorrect = 0

for row in test.to_numpy():
    if sum(a * row[:-1]) - b > -1 and row[-1] == 0:
        correctX += 1
    elif sum(a * row[:-1]) - b < -1 and row[-1] == 1:
        correctY += 1
    else:
        incorrect += 1
        
print(F"X correct = {correctX}")
print(F"Y correct = {correctY}")
print(F"Incorrect = {incorrect}")
print(F"Accuracry = {(correctX + correctY) / len(test.to_numpy())}")

X correct = 200
Y correct = 116
Incorrect = 27
Accuracry = 0.9212827988338192


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d49c4e00-e54d-42d3-8cda-6e1a56ded0ed' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>